In [1]:
import tiktoken
from sentence_transformers import SentenceTransformer

e:\D\Work\Projects\Medical Lab Bot\Test_Files\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd

df = pd.read_csv("../DATA/Scraped_Data/medical_tests_interpretation.csv")
df2 = pd.read_csv("../DATA/Scraped_Data/testing_scraped_content.csv")

In [3]:
def embeddingCreator(text):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    tokens = model.encode(text).tolist()
    return tokens

In [4]:
def Scrapper(df,source):
    df_tokens = pd.DataFrame(columns=["Test Name","tokens","Source","URL"])
    for i in range(len(df)):
        text = df.loc[i,"Description"]
        test_Name = df.loc[i,"Test Name"]
        Url = df.loc[i,"URL"]

        tokens = embeddingCreator(text)
        df_tokens = pd.concat([df_tokens, pd.DataFrame([{"Test Name":test_Name,"tokens":tokens, "Source":source,"URL":Url}])], ignore_index=True)
        print("Scrapped ",i)
    return df_tokens

def writeToFile(df_tokens, filename):
    df_tokens.to_csv(filename, index=False)

In [5]:
def CreateEmbedding(df,source,path):
    df['Description'] = df['Description'].fillna('')
    x_df = Scrapper(df,source)
    writeToFile(x_df,path)

In [6]:
CreateEmbedding(df,"medlinePlus.com","../DATA/Scraped_Data/medlinePlus_tokens.csv")
CreateEmbedding(df2,"testing.com","../DATA/Scraped_Data/testing_com_tokens.csv")

Scrapped  0
Scrapped  1
Scrapped  2
Scrapped  3
Scrapped  4
Scrapped  5
Scrapped  6
Scrapped  7
Scrapped  8
Scrapped  9
Scrapped  10
Scrapped  11
Scrapped  12
Scrapped  13
Scrapped  14
Scrapped  15
Scrapped  16
Scrapped  17
Scrapped  18
Scrapped  19
Scrapped  20
Scrapped  21
Scrapped  22
Scrapped  23
Scrapped  24
Scrapped  25
Scrapped  26
Scrapped  27
Scrapped  28
Scrapped  29
Scrapped  30
Scrapped  31
Scrapped  32
Scrapped  33
Scrapped  34
Scrapped  35
Scrapped  36
Scrapped  37
Scrapped  38
Scrapped  39
Scrapped  40
Scrapped  41
Scrapped  42
Scrapped  43
Scrapped  44
Scrapped  45
Scrapped  46
Scrapped  47
Scrapped  48
Scrapped  49
Scrapped  50
Scrapped  51
Scrapped  52
Scrapped  53
Scrapped  54
Scrapped  55
Scrapped  56
Scrapped  57
Scrapped  58
Scrapped  59
Scrapped  60
Scrapped  61
Scrapped  62
Scrapped  63
Scrapped  64
Scrapped  65
Scrapped  66
Scrapped  67
Scrapped  68
Scrapped  69
Scrapped  70
Scrapped  71
Scrapped  72
Scrapped  73
Scrapped  74
Scrapped  75
Scrapped  76
Scrapped 